# 🕵️ Food Safety 2


## 📝 This Assignment

I will continue my exploration of restaurant food safety scores for restaurants in San Francisco. The main goal for this project is to focus more on the analysis of the dataset, building on the data cleaning we have done earlier in Food Safety 1


In [21]:
import numpy as np
import pandas as pd

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

In [22]:
bus = pd.read_csv('data/bus.csv', encoding='ISO-8859-1').rename(columns={"business id column": "bid"})
bus['postal5'] = bus['postal_code'].str[:5]
ins = pd.read_csv('data/ins.csv')
ins['timestamp'] = pd.to_datetime(ins['date'], format='%m/%d/%Y %I:%M:%S %p')
ins['bid'] = ins['iid'].str.split("_", expand=True)[0].astype(int) 


<br/>

---

# 🔎 Section 1: Inspecting the Inspections

I'll start by looking again at the first 5 rows of `ins` to see what I'm working with.

In [23]:
ins.head(5)

,iid,date,score,type,timestamp,bid
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,2019-03-29,100010
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,2019-04-17,100017
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,2019-08-26,100017


## Part 1

To better understand how the scores have been allocated, i'll examine how the maximum score varies for each type of inspection. 

I'll create a `DataFrame` object `ins_score_by_type`, indexed by all the inspection types (e.g., New Construction, Routine - Unscheduled, etc.), with a single column named `max_score` containing the highest score received. Additionally, order `ins_score_by_type` by `max_score` in descending order. 

In [24]:
ins_score_by_type = ins.groupby('type')['score'].max().reset_index().rename(columns = {'score':'max_score'}).sort_values(by = 'max_score', ascending=False).set_index("type")
ins_score_by_type

,max_score
type,
Routine - Unscheduled,100
Administrative or Document Review,-1
Community Health Assessment,-1
Complaint,-1
Complaint Reinspection/Followup,-1
Foodborne Illness Investigation,-1
Multi-agency Investigation,-1
New Construction,-1
New Ownership,-1


<br/>

---

## Part 2


Given the variability of `ins['score']` observed in part 1, i'll examine the inspection scores `ins['score']` further.

In [25]:
ins['score'].value_counts().head()

score
-1      12632
 100     1993
 96      1681
 92      1260
 94      1250
Name: count, dtype: int64

There are a large number of inspections with a score of -1. These are probably missing values. I'll see what types of inspections have scores and which do not (score of -1). 

- First, define a new column `Missing Score` in `ins` where each row maps to the string `"Yes"` if the `score` for that business is -1 and `"No"` otherwise. 

- Then, use `groupby` to find the number of inspections for every combination of `type` and `Missing Score`. Store these values in a new column `Count`. 

- Finally, sort `ins_missing_score_group` by descending `Count`s. 
The result should be a `DataFrame` that looks like the one shown below.

In [26]:
ins['Missing Score'] = ins['score'].map(lambda x: 'Yes' if x == -1 else 'No')
ins_missing_score_group = (
    ins.groupby(["type", "Missing Score"])
    .size()
    .reset_index(name="Count")
    .sort_values(by="Count", ascending=False)
    .set_index(["type", "Missing Score"])
)
ins_missing_score_group

,,Count
type,Missing Score,
Routine - Unscheduled,No,14031
Reinspection/Followup,Yes,6439
New Ownership,Yes,1592
Complaint,Yes,1458
New Construction,Yes,994
Non-inspection site visit,Yes,811
New Ownership - Followup,Yes,499
Structural Inspection,Yes,394
Complaint Reinspection/Followup,Yes,227


<br/>

---

## Part 3


Using `groupby` to perform the analysis above gave me a `DataFrame` that wasn't the most readable at first glance. There are better ways to represent the information above that take advantage of the fact that we are looking at combinations of two variables. It's time to pivot.

Create a `DataFrame` that looks like the one below, and assign it to the variable `ins_missing_score_pivot`. 

I want to use the `pivot_table` method of the `DataFrame` class, which I can read about in the `pivot_table` [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html). 

- Once I create `ins_missing_score_pivot`, add another column titled `Percentage Missing`, which contains the proportion of missing scores within each `type`. 

- Then, sort `ins_missing_score_pivot` in ascending order of `Percentage Missing`. Reassign the sorted `DataFrame` back to `ins_missing_score_pivot`.


In [27]:
ins_missing_score_pivot = ins_missing_score_group.pivot_table(
    index = 'type',
    columns = 'Missing Score',
    values = 'Count',
    fill_value = 0
)
ins_missing_score_pivot['Percentage Missing'] = ins_missing_score_pivot['Yes'] / (ins_missing_score_pivot['No'] + ins_missing_score_pivot['Yes'])
ins_missing_score_pivot = ins_missing_score_pivot.sort_values(by = 'Percentage Missing', ascending = True)
ins_missing_score_pivot

Missing Score,No,Yes,Percentage Missing
type,,,
Routine - Unscheduled,14031.0,46.0,0.003268
Administrative or Document Review,0.0,4.0,1.000000
Community Health Assessment,0.0,1.0,1.000000
Complaint,0.0,1458.0,1.000000
Complaint Reinspection/Followup,0.0,227.0,1.000000
Foodborne Illness Investigation,0.0,115.0,1.000000
Multi-agency Investigation,0.0,3.0,1.000000
New Construction,0.0,994.0,1.000000
New Ownership,0.0,1592.0,1.000000


Notice that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections. Also, it is reasonable for inspection types such as `New Ownership` and `Complaint` to have no associated inspection scores, but you might be curious why there are no inspection scores for the `Reinspection/Followup` inspection type.

<br/>

---

# Section 2: Joining Data Across Tables

<br/>

--- 

## Part 1

I'll figure out which restaurants had the lowest scores. Before I proceed, I'll filter out missing scores from `ins` so that negative scores don't influence our results. 

In [28]:
ins = ins[ins["score"] > 0]

I'll start by creating a new `DataFrame` called `ins_named`. `ins_named` should be exactly the same as `ins`, except that it should have the name and address of every business, as determined by the `bus` `DataFrame`. 

In [29]:
ins_named = ins.merge(bus[['bid', 'name', 'address']], on = 'bid', how = 'left')
ins_named.head()

,iid,date,score,type,timestamp,bid,Missing Score,name,address
0,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010,No,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B
1,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017,No,AMICI'S EAST COAST PIZZERIA,475 06th St
2,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,2019-05-20,100041,No,UNCLE LEE CAFE,3608 BALBOA ST
3,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,2019-04-25,100055,No,Twirl and Dip,335 Martin Luther King Jr. Dr
4,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,2019-09-12,100055,No,Twirl and Dip,335 Martin Luther King Jr. Dr


<br/>

--- 

## Part 2

I'll look at the 20 businesses in `ins_named` with the lowest scores. Order `ins_named` by each business's minimum score in ascending order. I'll use the business names in ascending order to break ties.

In [30]:
twenty_lowest_scoring = ins_named[['bid', 'name', 'score']].groupby('bid').min().rename(columns = {'score':'min score'}).sort_values('min score', ascending = True).head(20)
twenty_lowest_scoring

,name,min score
bid,,
86718,Lollipot,45
1154,SUNFLOWER RESTAURANT,46
10877,CHINA FIRST INC.,48
67237,La Jefa,51
84590,Chaat Corner,54
71310,Golden King Vietnamese Restaurant,55
91843,Hello Sandwich & Noodle,55
59828,Tip Top Market,55
66961,Mi Tierra Market,57


<br/>

--- 
## Part 3
I'll do some more interesting analysis with my lowest score calculations. In the cell below, assign `worst_3_inspection_restaurants` to a two-column `DataFrame` with 15 rows.

- One column is the `name` of each business.

- The other column is a modified average inspection score of each business called `lowest 3 average`. 

    - To calculate `lowest 3 average`, find the average of each business's **three lowest inspection scores**. 

    - If a business has less than three inspection scores, take the average of all of its inspection scores (i.e., either one or two scores). 

- Finally, assign `worst_3_inspection_restaurants` to a `DataFrame` of the 15 rows with the lowest `lowest 3 average`, sorted by `lowest 3 average` ascending.  

`worst_3_inspection_restaurants` should look like the one below.


In [37]:
ins_sorted = ins_named.sort_values(by=["bid", "score"])
lowest_3_avg = (
    ins_sorted.groupby('bid')
    .apply(lambda group: group.sort_values('score', ascending = True).head(3)['score'].mean())
    .reset_index(name = 'lowest 3 average')
)
unique_names = ins_named[['bid', 'name']].drop_duplicates(subset = 'bid')
worst_3_inspection_restaurants = (
    lowest_3_avg.merge(
        unique_names, 
        on = 'bid', 
        how = 'left'
    )
    .set_index('bid')
    .sort_values(by = 'lowest 3 average', ascending=True)
    .head(15)
)
worst_3_inspection_restaurants = worst_3_inspection_restaurants[['name', 'lowest 3 average']]
worst_3_inspection_restaurants

/var/folders/4q/b0zn7z6d7v12h1j1p66g5tf80000gn/T/ipykernel_49335/1378662910.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ins_sorted.groupby('bid')


,name,lowest 3 average
bid,,
84590,Chaat Corner,54.0
90622,Taqueria Lolita,57.0
94351,VBowls LLC,58.0
69282,New Jumbo Seafood Restaurant,60.5
1154,SUNFLOWER RESTAURANT,63.5
93150,Chez Beesen,64.0
93502,Smoky Man,64.0
98995,Vallarta's Taco Bar,64.0
39776,Duc Loi Supermarket,64.0


<!-- END QUESTION -->

<br/>

---

# Section 3 Missing Inspections

With my inspection data, I am given the `type` of each inspection. These categories were lightly investigated in Question 1, centered on the number of missing scores within each `type`. Since the `timestamp` and `score` for each inspection are also provided, I can do a more interesting analysis relating the `score` and `timestamp` of specific types of inspections. 

Specifically, in Part 3, I am interested in the possible relationship between inspections of the `type` "Routine - Unscheduled" and "Reinspection/Followup" (the two most frequent inspection types in our dataset). We might guess that a follow-up ("Reinspection/Followup") inspection occurs more frequently when an initial ("Routine - Unscheduled") inspection receives a low score. To confirm this hunch, I'll investigate the rate of follow-up inspections for different initial scores. To simplify your analysis, I have provided a new `DataFrame` (`reinspections`). 

- `reinspections` contains every "Routine - Unscheduled" inspection, along with the relevant `bid` and `name` associated with the initial inspection. 
- `routine timestamp` indicates when the initial inspection occurred. 
- `routine score` is the score that the initial inspection received. 
- `day difference` is the number of days between the initial inspection and a follow-up inspection if done within one year. 
    
Some initial inspections did not have any follow-up inspections within one year. In these cases, `day difference` is assigned a filler value of -1.

Run the cell below to load in `reinspections`.

In [32]:
reinspections = pd.read_csv('data/reinspections.csv')
reinspections

,bid,routine timestamp,routine score,name,day difference
0,19,2017-12-11,94,Nrgize Lifestyle Cafe,-1.0
1,19,2018-06-07,96,Nrgize Lifestyle Cafe,-1.0
2,24,2016-10-05,98,OMNI S.F. Hotel - 2nd Floor Pantry,-1.0
3,24,2017-11-01,98,OMNI S.F. Hotel - 2nd Floor Pantry,-1.0
4,24,2019-05-01,98,OMNI S.F. Hotel - 2nd Floor Pantry,15.0
...,...,...,...,...,...
14072,101853,2019-09-18,100,INDIAN MARKET AND LIQUOR,-1.0
14073,102067,2019-09-11,100,Humphry Slocombe Ice Cream,-1.0
14074,102257,2019-09-25,94,THE SLICE,7.0
14075,102336,2019-09-19,82,RED HILL STATION,1.0


<br/>

--- 
## Part 1
First, I created a new `Boolean` column `recent reinspection?` that indicates whether a follow-up inspection occurred within 62 days inclusive (~2 months) of an initial inspection. 

In [33]:
reinspections['recent reinspection?'] = reinspections['day difference'].between(0, 62)

reinspections

,bid,routine timestamp,routine score,name,day difference,recent reinspection?
0,19,2017-12-11,94,Nrgize Lifestyle Cafe,-1.0,False
1,19,2018-06-07,96,Nrgize Lifestyle Cafe,-1.0,False
2,24,2016-10-05,98,OMNI S.F. Hotel - 2nd Floor Pantry,-1.0,False
3,24,2017-11-01,98,OMNI S.F. Hotel - 2nd Floor Pantry,-1.0,False
4,24,2019-05-01,98,OMNI S.F. Hotel - 2nd Floor Pantry,15.0,True
...,...,...,...,...,...,...
14072,101853,2019-09-18,100,INDIAN MARKET AND LIQUOR,-1.0,False
14073,102067,2019-09-11,100,Humphry Slocombe Ice Cream,-1.0,False
14074,102257,2019-09-25,94,THE SLICE,7.0,True
14075,102336,2019-09-19,82,RED HILL STATION,1.0,True


<br/>

--- 
## Part 2
To simplify my analysis, I'll assign `routine score`s to buckets. Buckets are similar to the bins of a histogram. Each bucket contains all scores that fall in a particular range.

Below I have defined the function `bucketify`. Use `bucketify` to create a new column in `reinspections` called `score buckets` that **maps** the score of an initial inspection to one of these predefined buckets.

In [34]:
def bucketify(score):
    if score < 65: 
        return '0 - 65'
    elif score < 70:
        return '65 - 69'
    elif score < 75:
        return '70 - 74'
    elif score < 80:
        return '75 - 79'
    elif score < 85:
        return '80 - 84'
    elif score < 90:
        return '85 - 89'
    elif score < 95:
        return '90 - 94'
    else:
        return '95 - 100'
        
reinspections['score buckets'] = reinspections['routine score'].map(lambda x: bucketify(x))

reinspections

,bid,routine timestamp,routine score,name,day difference,recent reinspection?,score buckets
0,19,2017-12-11,94,Nrgize Lifestyle Cafe,-1.0,False,90 - 94
1,19,2018-06-07,96,Nrgize Lifestyle Cafe,-1.0,False,95 - 100
2,24,2016-10-05,98,OMNI S.F. Hotel - 2nd Floor Pantry,-1.0,False,95 - 100
3,24,2017-11-01,98,OMNI S.F. Hotel - 2nd Floor Pantry,-1.0,False,95 - 100
4,24,2019-05-01,98,OMNI S.F. Hotel - 2nd Floor Pantry,15.0,True,95 - 100
...,...,...,...,...,...,...,...
14072,101853,2019-09-18,100,INDIAN MARKET AND LIQUOR,-1.0,False,95 - 100
14073,102067,2019-09-11,100,Humphry Slocombe Ice Cream,-1.0,False,95 - 100
14074,102257,2019-09-25,94,THE SLICE,7.0,True,90 - 94
14075,102336,2019-09-19,82,RED HILL STATION,1.0,True,80 - 84


<br/>

--- 
## Part 3
Before I complete my analysis, I'll remove all rows whose `score buckets` contain less than 125 rows. Assign `reinspection_filtered` to this new `DataFrame`. 

In [35]:
bucket_counts = reinspections['score buckets'].value_counts()
valid_buckets = bucket_counts[bucket_counts >= 125].index
reinspections_filtered = reinspections[reinspections['score buckets'].isin(valid_buckets)]
reinspections_filtered

,bid,routine timestamp,routine score,name,day difference,recent reinspection?,score buckets
0,19,2017-12-11,94,Nrgize Lifestyle Cafe,-1.0,False,90 - 94
1,19,2018-06-07,96,Nrgize Lifestyle Cafe,-1.0,False,95 - 100
2,24,2016-10-05,98,OMNI S.F. Hotel - 2nd Floor Pantry,-1.0,False,95 - 100
3,24,2017-11-01,98,OMNI S.F. Hotel - 2nd Floor Pantry,-1.0,False,95 - 100
4,24,2019-05-01,98,OMNI S.F. Hotel - 2nd Floor Pantry,15.0,True,95 - 100
...,...,...,...,...,...,...,...
14072,101853,2019-09-18,100,INDIAN MARKET AND LIQUOR,-1.0,False,95 - 100
14073,102067,2019-09-11,100,Humphry Slocombe Ice Cream,-1.0,False,95 - 100
14074,102257,2019-09-25,94,THE SLICE,7.0,True,90 - 94
14075,102336,2019-09-19,82,RED HILL STATION,1.0,True,80 - 84


<br/>

--- 
## Part 4

To conclude my analysis, I'll use `resinpsections_filtered` to generate a `DataFrame` with the **proportion** of initial inspections within each bucket that were reinspected within 62 days, along with the total **count** of initial inspections included in each bucket. I'll sort this `DataFrame` by ascending counts. Assign this new `DataFrame` to `reinspection_proportions`.

In [36]:
reinspection_proportions = reinspections_filtered.groupby('score buckets').agg(
    proportion=('recent reinspection?', 'mean'),
    count=('score buckets', 'size')
)
reinspection_proportions.columns = pd.MultiIndex.from_tuples([
    ('recent reinspection?', 'proportion'),
    ('recent reinspection?', 'count')
])
reinspection_proportions = reinspection_proportions.sort_values(by = ('recent reinspection?', 'count'), ascending = True)
reinspection_proportions

recent reinspection?      
                        proportion count
score buckets                           
70 - 74                   0.407821   358
75 - 79                   0.405248   686
80 - 84                   0.371889  1366
85 - 89                   0.304811  2536
90 - 94                   0.171498  4140
95 - 100                  0.036601  4754